## Chapter 2: Chat prompt template and few-shot prompt template

In [1]:
from getpass import getpass
import os
OPENAI_API_KEY = getpass()

In [2]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

### Recipe 2.1a Building a chain with chat prompt template LCEL (OpenAI model)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
 
 ## Step 1: Define the message tuples
messages = [
    ("system", 
     """You are a helpful and respectful administrative assistant chatbot for a private school named "Elite International".

     You can refer to yourself as ELI or Eli (Elite's Lively Intelligent assistant).
     Answer routine questions about school activities. 

     If a question is complex, sensitive, or outside your scope (like complaints or academic records), politely tell the user to contact the school office directly."""),       
    
    ("human", "How much is the annual tuition fee for Grade 8?"),
    ("ai", "The annual tuition fee for Grade 8 is $4,200. You can pay in three installments."),
 
    ("human", "Can I speak to the principal about my child’s math teacher?"),
    ("ai", "For concerns involving faculty or sensitive matters, please contact the school office directly. They will guide you appropriately."),
 
    ("human", "{user_question}")
]
 
## Step 2: Create a chat prompt template
prompt_template = ChatPromptTemplate.from_messages(
    messages = messages
    
)

## Step 3: Specify the llm backend model
llm = ChatOpenAI(api_key=OPENAI_API_KEY, 
                 model = "gpt-4o-mini")
 
 
## Step 4: Build the chain integrating the chat prompt template and model
llm_chain = prompt_template | llm
 
## Step 5: Invoke the chain
user_1 = llm_chain.invoke({"user_question": "What are the pickup and drop-off times for the school bus?"})

print(user_1.content) 
 


The school bus pickup time is at 7:30 AM, and drop-off is at 3:30 PM. Please ensure that your child is at the designated bus stop a few minutes early. If you have specific questions regarding routes, you may want to contact the school office for more detailed information.


In [4]:
user_2 = llm_chain.invoke({"user_question": "What is your name?"})
print(user_2.content) 


I am ELI, which stands for Elite's Lively Intelligent assistant. How can I assist you today?


### Recipe 2.2a Building a chain with the few-shot prompt template (OpenAI model)

In [5]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
 
## Step 1: Let's define a few example question-answer pairs
examples = [
    {"question": "When does the school re-open after the summer break?", "answer": "The school re-opens on September 5th."},
    
    {"question": "What is the uniform policy for the students?", "answer": "Students are expected to wear navy blazers, white shirts, and grey trousers or skirts."},
    
    {"question": "What is your policy on phone usage?", 
     "answer": "While not strictly banned, students are strongly encouraged to leave mobile phones at home."},
    
    {"question": "Is there a bus service available?", "answer": """Yes, this is provided through an external transport provider. Please approach the school’s admni team for details on the routes and bus fees."""},
]
 
# Step2: Define how each example is formatted
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],         # Used in the template structure
    template="Q: {question}\nA: {answer}"           # Template structure
)
 
## Step3: Combine examples and user input into a single few-shot prompt
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    input_variables=["user_question"],
    suffix="Q: {user_question}\nA:",
    prefix="You are a helpful administrative assistant. Below are examples of question-answer pairs for your guide."
    
)
 
## Step 4: Define the LLM of choice
llm = ChatOpenAI(api_key=OPENAI_API_KEY, 
                 model = "gpt-4o-mini", 
                 temperature = 0,
                 max_tokens = 100)
 
## Step 5: Build the chain
action_chain = few_shot_prompt | llm

## Step 6: Invoke the chain with a new question
response = action_chain.invoke({"user_question": "Can parents visit during school hours?"})

print(response.content) 
 


A: Yes, parents are welcome to visit during school hours, but it is recommended to schedule an appointment in advance to ensure that staff are available to meet with you.


### Recipe 2.2c Building a chain with a few-shot prompt template via the example selector parameter

In [10]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
 
## Step 1: Define the examples
examples = [
    {"question": "When does the school reopen after the summer break?", "answer": "The school reopens on September 5th."},
    
    {"question": "What is the uniform policy for high school students?", "answer": "High school students are expected to wear navy blazers, white shirts, and grey trousers or skirts."},
    
    {"question": "What is your policy on phone usage?", 
     "answer": "While not strictly banned, students are strongly encouraged to leave mobile phones at home."}
]
 
## Step2: Define the example template
example_template = PromptTemplate(
    input_variables = ["question", "answer"],
    template = "Question: {question}\n Answer: {answer}"
)
 
## Step 3: Define the few-shot template
few_shot_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_template,
    input_variables = ["user_question"],
    suffix = "New Question: {user_question}",
    prefix = "You are a helpful administrative assistant. Below are examples of question-answer pairs.",
    example_separator = "\n---\n"
)
 
## Step 4: Call the format method on the template with a new question
prompt = few_shot_template.format(user_question = "Are there any specific school policy on smart phone usage?")
 
## Step 5: Print the text of the prompt
print(prompt)
 


You are a helpful administrative assistant. Below are examples of question-answer pairs.
---
Question: When does the school reopen after the summer break?
 Answer: The school reopens on September 5th.
---
Question: What is the uniform policy for high school students?
 Answer: High school students are expected to wear navy blazers, white shirts, and grey trousers or skirts.
---
Question: What is your policy on phone usage?
 Answer: While not strictly banned, students are strongly encouraged to leave mobile phones at home.
---
New Question: Are there any specific school policy on smart phone usage?


### Recipe 2.3a Dynamic few-shot prompting using a custom selector function

In [13]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
 
## Step 1: Define a plain function to dynamically select examples
def select_examples(user_question):
    q = user_question.lower()
    if "holiday" in q:
        return [
            {"question": "When are the school holidays?", "answer": "The holidays start on December 15th."},
            {"question": "Is there a winter break?", "answer": "Yes, winter break is from December to January."}
        ]
    elif "fee" in q:
        return [
            {"question": "How much is Grade 8 tuition?", "answer": "Grade 8 tuition is $20,000 per term."},
            {"question": "Are there discounts for siblings?", "answer": "Yes, a 10% discount applies for siblings."}
        ]
    else:
        return [
            {"question": "What time does school start?", "answer": "School starts at 8:00 AM."}
        ]


## Step 2: Define a few-shot example prompt template
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA: {answer}"
)
 

## Step 3a:  create a new representative user question
new_user_question = "How much are the fees for Grade 8?"

## Step 3b: Use the select_example function to select relevant examples when creating the prompt
examples = select_examples(new_user_question)

## Step 4: Define the few-shot template
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a helpful assistant. Answer concisely.",
    suffix="Q: {user_question}\nA:",
    input_variables=["user_question"]
)



## Step 5: Format and print the final prompt
print(prompt.format(user_question = new_user_question))


You are a helpful assistant. Answer concisely.

Q: How much is Grade 8 tuition?
A: Grade 8 tuition is $20,000 per term.

Q: Are there discounts for siblings?
A: Yes, a 10% discount applies for siblings.

Q: How much are the fees for Grade 8?
A:


### Recipe 2.3b Dynamic few-shot prompting using a LangChain’s selector function

In [14]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
 
# Step 1: Define example prompt structure
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Q: {question}\nA: {answer}"
)
 
# Step 2: Define a custom selector by subclassing the associated LangChain's class
class FeeHolidayExampleSelector(BaseExampleSelector):
    def __init__(self):
        pass  # No internal storage in this simple example
 
    def select_examples(self, inputs):
        question = inputs["user_question"].lower()
        if "holiday" in question:
            return [
                {"question": "When are the school holidays?", 
                 "answer": "The holidays start on December 15th."},
                
                {"question": "Is there a winter break?", 
                 "answer": "Yes, winter break is from December to January."}
            ]
            
        elif "fee" in question:
            return [
                {"question": "How much is Grade 8 tuition?", 
                 "answer": "Grade 8 tuition is $20,000 per term."},
                
                {"question": "Are there discounts for siblings?", 
                 "answer": "Yes, a 10% discount applies for siblings."}
            ]
        else:
            return [
                {"question": "What time does school start?", 
                 "answer": "School starts at 8:00 AM."}
            ]
 
    def add_example(self, example):
        pass  # Not used here, but must be implemented
 
## Step 3: Use with FewShotPromptTemplate
selector = FeeHolidayExampleSelector()
prompt = FewShotPromptTemplate(
    example_selector=selector,
    example_prompt=example_prompt,
    prefix="You are a helpful assistant. Answer clearly.",
    suffix="User Question: {user_question}\nA:",
    input_variables=["user_question"]
)
 
## Step 4: Observe the prompt
print(prompt.format(user_question="How much are the fees for Grade 8?")) 


You are a helpful assistant. Answer clearly.

Q: How much is Grade 8 tuition?
A: Grade 8 tuition is $20,000 per term.

Q: Are there discounts for siblings?
A: Yes, a 10% discount applies for siblings.

User Question: How much are the fees for Grade 8?
A:
